In [13]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from PIL import Image
from keras.models import model_from_json
from keras.optimizers import Adam

import tensorflow as tf

In [14]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
# todo
#     print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    
    
    #print("Patient ID", ds.PatientID)
    #print("Modality", ds.Modality)
    #print("Patient Position", ds.PatientPosition)
    #print("Body Part", ds.BodyPartExamined)
    
    
    if ds.Modality == 'DX':
        if ds.PatientPosition == 'PA' or ds.PatientPosition == 'AP': 
            if ds.BodyPartExamined == 'CHEST':
                return img
            else:
                return None
        else: 
            return None
    else: 
        return None
        
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    
    plt.imsave('test.png', img) # convert pixel array into image png format
    
    # Load the image into PIL format passing image_size
    img = tf.keras.preprocessing.image.load_img('test.png', grayscale=False, color_mode="rgb", target_size=img_size, interpolation="nearest")
    
    img = keras.preprocessing.image.img_to_array(img) # Convert to array 
    
    proc_img = (img - img_mean) / img_std # normalize image subtracting mean and dividing it by standard deviation 
    
    proc_img = np.expand_dims(proc_img, axis=0)  # Convert single image to a batch with dimension 1.
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    
    # read model arch into json file. 
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()

    # load model from json     
    model = model_from_json(loaded_model_json)
    
    #load weight
    model.load_weights(weight_path)
    
    # define model parameters
    optimizer = Adam(lr=1e-4)
    loss = 'binary_crossentropy'
    metrics = ['binary_accuracy']
    
    # Compile model with optimizer, loss and metrics 
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo
    
    # Run prediction
    prediction = model.predict(img)
    
    # Compare prediction with threshold
    if prediction > thresh:
        prediction = "Pneumonia detected"
    else:     
        prediction = "Pneumonia not present"
    
    return prediction 

In [15]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json"
weight_path = "xray_class_my_model.best.hdf5"

IMG_SIZE=(224,224) # This might be different if you did not use vgg16
img_mean = np.array([0.485, 0.456, 0.406])
img_std = np.array([0.229, 0.224, 0.225])

my_model = load_model(model_path, weight_path)

thresh = 0.26 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print("Prediction: ", pred)

Prediction:  Pneumonia detected
Prediction:  Pneumonia not present
Prediction:  Pneumonia detected
